
# Clasificación de proyectos exitosos (Cali, CO) a partir del **dataset unido** (Inmuebles + Proyectos)

**Objetivo:** identificar y explicar qué proyectos son *exitosos* según su **velocidad de venta**.  
Trabajaremos con tu archivo `Base Proyectos.xlsx`, uniendo las hojas **Inmuebles** y **Proyectos**, calculando métricas de velocidad y entrenando un modelo explicativo de patrones.



## 1. Preparación


In [ ]:

# Requisitos: pandas, numpy, scikit-learn, matplotlib
import pandas as pd
import numpy as np
import re
from pathlib import Path

import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor



## 2. Funciones auxiliares


In [ ]:

def to_num(x):
    """Convierte strings con $/separadores a float; respeta NaN."""
    if pd.isna(x): return np.nan
    if isinstance(x, (int, float, np.integer, np.floating)): return float(x)
    s = str(x)
    s = s.replace("\xa0"," ").replace("$","").replace("COP","").replace("cop","")
    s = s.replace(".", "").replace(",", ".")
    s = re.sub(r"[^0-9.\-]", "", s)
    try:
        return float(s)
    except:
        return np.nan

def find_col(df, required_all=None, required_any=None, forbidden_any=None):
    """Encuentra columnas por reglas de inclusión/exclusión en el nombre."""
    required_all = required_all or []
    required_any = required_any or []
    forbidden_any = forbidden_any or []
    cands = []
    for c in df.columns:
        cl = c.lower()
        if any(f in cl for f in forbidden_any):
            continue
        if not all(r in cl for r in required_all):
            continue
        if required_any and not any(r in cl for r in required_any):
            continue
        cands.append(c)
    return cands

def winsorize(s, p_low=0.01, p_high=0.99):
    if s.dropna().empty: return s
    lo = s.quantile(p_low); hi = s.quantile(p_high)
    return s.clip(lower=lo, upper=hi)

def summarize(df, name):
    print(f"{name}: {df.shape[0]} filas, {df.shape[1]} columnas")
    display(df.head(3))



## 3. Cargar datos (Excel) y vista rápida


In [ ]:

xlsx_path = Path('Base Proyectos.xlsx')
xls = pd.ExcelFile(xlsx_path)
inm = pd.read_excel(xls, 'Inmuebles')
pry = pd.read_excel(xls, 'Proyectos')

summarize(inm, "Inmuebles")
summarize(pry, "Proyectos")



## 4. Unir Inmuebles ↔ Proyectos
Preferimos unir por **código de proyecto** y, si no está, por **nombre**.


In [ ]:

# Detectar llaves
key_inm = None
for k in ['Cod Proyecto', 'Codigo Proyecto', 'Código Proyecto']:
    if k in inm.columns:
        key_inm = k; break
if key_inm is None and 'Proyecto' in inm.columns:
    key_inm = 'Proyecto'

key_pry = None
for k in ['Codigo Proyecto', 'Cod Proyecto', 'Código Proyecto']:
    if k in pry.columns:
        key_pry = k; break
if key_pry is None and 'Proyecto' in pry.columns:
    key_pry = 'Proyecto'

print("Llaves detectadas -> Inmuebles:", key_inm, "| Proyectos:", key_pry)

# Unión (left sobre Inmuebles para enriquecer unidades con atributos del proyecto)
inm_join = inm.merge(pry, left_on=key_inm, right_on=key_pry, how='left', suffixes=('','_pry'))
summarize(inm_join, "Inmuebles + atributos de proyecto (nivel unidad)")



## 5. Ingeniería de variables (unidad) y agregación a nivel proyecto
Calculamos métricas por unidad (precio, área, precio/m², alcobas, etc.) y luego **agregamos por proyecto** (medianas, proporciones) para obtener un dataset **a nivel de proyecto** que servirá para clasificar el éxito y modelar patrones.


In [ ]:

# Columnas típicas (ajusta si los nombres difieren)
col_precio_u = next((c for c in inm_join.columns if c.lower().strip() in ['precio', 'valor']), None)
col_area_u   = next((c for c in inm_join.columns if 'área' in c.lower() or 'area' in c.lower()), None)
col_alcobas  = next((c for c in inm_join.columns if 'alcobas' in c.lower() or 'habitaciones' in c.lower()), None)
col_banos_c  = next((c for c in inm_join.columns if 'baños completos' in c.lower() or 'banos completos' in c.lower()), None)
col_garajes  = next((c for c in inm_join.columns if 'garajes' in c.lower() or 'numero de garajes' in c.lower()), None)
col_tipo     = next((c for c in inm_join.columns if 'tipo inmueble' in c.lower()), None)
col_vis      = next((c for c in inm_join.columns if 'tipo vis' in c.lower() or (c.lower().strip()=='tipo vis')), None)

# Limpieza numérica unidad
if col_precio_u: inm_join['_precio'] = inm_join[col_precio_u].apply(to_num)
if col_area_u:   inm_join['_area']   = pd.to_numeric(inm_join[col_area_u], errors='coerce')

# Precio por m2 y winsor
if col_precio_u and col_area_u:
    inm_join['_ppm2'] = inm_join['_precio'] / inm_join['_area']
    inm_join['_ppm2'] = winsorize(inm_join['_ppm2'])

# Agregados por proyecto (proyecto del lado Inmuebles)
proj_key = key_inm
grp = inm_join.groupby(proj_key)

agg_dict = {}

if col_area_u:
    agg_dict['_area'] = ['median','mean','min','max']
if col_precio_u:
    agg_dict['_precio'] = ['median','mean']
if col_precio_u and col_area_u:
    agg_dict['_ppm2'] = ['median','mean','min','max']

if col_alcobas:
    agg_dict[col_alcobas] = ['median','mean']
if col_banos_c:
    agg_dict[col_banos_c] = ['median','mean']
if col_garajes:
    agg_dict[col_garajes] = ['median','mean']

# Conteo de unidades por proyecto
agg_dict[proj_key] = ['count']

proj_feat_u = grp.agg(agg_dict)
# Aplanar columnas
proj_feat_u.columns = ['{}_{}'.format(k, stat) if k!=proj_key else 'n_unidades' for (k, stat) in proj_feat_u.columns]
proj_feat_u = proj_feat_u.reset_index()

summarize(proj_feat_u, "Features agregadas desde unidades (nivel proyecto)")



## 6. Métricas de **velocidad** de venta (nivel proyecto)

- **Meses para agotar** = `Un. Disponible Proyecto` / `Ventas promedio (UNIDADES/mes)`  
- **Tiempo total estimado** = `meses_desde_inicio` + `meses_para_agotar` (si hay `Fecha Inicio`)


In [ ]:

prj = pry.copy()

# Detectar columnas clave en Proyectos
col_un_disp = next((c for c in prj.columns if 'dispon' in c.lower() and 'un' in c.lower()), None)

# Ventas promedio en UNIDADES/mes (evita columnas en $)
col_ventas_un = None
for c in prj.columns:
    cl = c.lower().replace('.', '').replace('  ',' ')
    if ('venta' in cl or 'ventas' in cl) and 'mes' in cl and (' un' in cl or 'unid' in cl or 'unidad' in cl):
        if '$' not in c:
            col_ventas_un = c; break
if not col_ventas_un:
    # Fallback: 'Capacidad Ventas'
    col_ventas_un = next((c for c in prj.columns if 'capacidad' in c.lower() and 'venta' in c.lower()), None)

col_tot_un   = next((c for c in prj.columns if ('tot' in c.lower() or 'total' in c.lower()) and 'proyecto' in c.lower() and ('un' in c.lower() or 'unidad' in c.lower())), None)
col_precio_m2= next((c for c in prj.columns if 'm2' in c.lower() and 'prom' in c.lower()), None)
col_precio_p = next((c for c in prj.columns if 'precio prom' in c.lower() and 'm2' not in c.lower()), None)
col_area_p   = next((c for c in prj.columns if 'area prom' in c.lower()), None)
col_zona     = 'Zona'      if 'Zona' in prj.columns else None
col_subzona  = next((c for c in prj.columns if 'sub zona' in c.lower() or 'subzona' in c.lower()), None)
col_barrio   = next((c for c in prj.columns if 'barrio' in c.lower()), None)
col_estrato  = 'Estrato'   if 'Estrato' in prj.columns else None
col_estado   = next((c for c in prj.columns if 'estado etapas' in c.lower() or (c.lower().startswith('estado') and 'etapa' in c.lower())), None)
col_codigo   = next((c for c in prj.columns if 'codigo proyecto' in c.lower()), None)
col_nombre   = 'Proyecto' if 'Proyecto' in prj.columns else None
col_fecha_ini= next((c for c in prj.columns if 'fecha inicio' in c.lower()), None)

print("Detectado -> UnDisp:", col_un_disp, "| Ventas UN/mes:", col_ventas_un, "| Fecha Inicio:", col_fecha_ini)

# Numéricos
for c in [col_un_disp, col_ventas_un, col_tot_un, col_precio_m2, col_precio_p, col_area_p, col_estrato]:
    if c and c in prj.columns:
        prj[c+"_num"] = prj[c].apply(to_num)

# Meses para agotar
if col_un_disp and col_ventas_un:
    prj['meses_para_agotar'] = prj[col_un_disp+"_num"] / np.clip(prj[col_ventas_un+"_num"], 0.1, None)
else:
    prj['meses_para_agotar'] = np.nan

# Meses desde inicio y tiempo total estimado
if col_fecha_ini and col_fecha_ini in prj.columns:
    prj['_fecha_ini'] = pd.to_datetime(prj[col_fecha_ini], errors='coerce')
    today = pd.Timestamp.today().normalize()
    prj['meses_desde_inicio'] = ((today - prj['_fecha_ini']).dt.days / 30.4375).clip(lower=0)
else:
    prj['meses_desde_inicio'] = np.nan

prj['tiempo_total_estimado_meses'] = prj['meses_desde_inicio'] + prj['meses_para_agotar']

summarize(prj[[col_nombre, col_zona, col_barrio, 'meses_para_agotar', 'tiempo_total_estimado_meses']].head(10), "Preview velocidad (proyecto)")



## 7. Dataset unido **a nivel proyecto** (features + targets)
Combinamos **agregados de unidades** + **campos de proyectos** + **targets**.


In [ ]:

# Merge de features agregadas (desde unidades) con proyectos (por código o nombre)
key_left = proj_key
key_right = key_pry

proj_ds = proj_feat_u.merge(prj, left_on=key_left, right_on=key_right, how='left', suffixes=('','_pry2'))

# Selección final de columnas
keep_cols = [c for c in [
    key_left, col_nombre, col_codigo, col_zona, col_subzona, col_barrio, col_estrato,
    col_estado, col_precio_m2, col_precio_p, col_area_p, col_tot_un,
    # agregados desde unidades
    '_area_median','_area_mean','_ppm2_median','_ppm2_mean','_precio_median','_precio_mean',
    (col_alcobas + '_median') if col_alcobas else None,
    (col_garajes + '_median') if col_garajes else None,
    'n_unidades',
    # targets
    'meses_para_agotar','meses_desde_inicio','tiempo_total_estimado_meses'
] if c and (c in proj_ds.columns)]

proj_ds = proj_ds[keep_cols].copy()

# Limpieza de numéricos
for c in [col_precio_m2, col_precio_p, col_area_p, col_tot_un, col_estrato,
          '_area_median','_area_mean','_ppm2_median','_ppm2_mean','_precio_median','_precio_mean']:
    if c in proj_ds.columns:
        proj_ds[c] = proj_ds[c].apply(to_num)

summarize(proj_ds, "Dataset unido a nivel proyecto (para clasificación)")

# Guardar un CSV de trabajo
out_path = Path('dataset_unido_proyecto.csv')
proj_ds.to_csv(out_path, index=False)
print("Guardado:", out_path.resolve())



## 8. Reglas de **clasificación de éxito**
Usaremos dos enfoques (elige el que prefieras para negocio):

1. **Velocidad actual (Meses para agotar):** Exitoso = Q1 (25% más rápido).  
2. **Tiempo total estimado (desde inicio):** Exitoso = Q1 del tiempo total.


In [ ]:

ds = proj_ds.copy()

# Clasificación por meses_para_agotar
valid = ds['meses_para_agotar'].dropna()
q1_mpa = valid.quantile(0.25); q3_mpa = valid.quantile(0.75)

def lab_mpa(x):
    if pd.isna(x): return np.nan
    if x <= q1_mpa: return "Exitoso (rápido)"
    if x >= q3_mpa: return "Lento"
    return "Medio"

ds['clasif_meses_para_agotar'] = ds['meses_para_agotar'].apply(lab_mpa)

# Clasificación por tiempo total estimado
valid2 = ds['tiempo_total_estimado_meses'].dropna()
if not valid2.empty:
    q1_tot = valid2.quantile(0.25); q3_tot = valid2.quantile(0.75)
    def lab_tot(x):
        if pd.isna(x): return np.nan
        if x <= q1_tot: return "Exitoso (tiempo total)"
        if x >= q3_tot: return "Lento (tiempo total)"
        return "Medio (tiempo total)"
    ds['clasif_tiempo_total'] = ds['tiempo_total_estimado_meses'].apply(lab_tot)

# Percentiles por Zona / Estrato (comparación en segmento)
if 'Zona' in ds.columns:
    ds['pct_en_zona_mpa'] = ds.groupby('Zona')['meses_para_agotar'].rank(pct=True) * 100.0
if 'Estrato' in ds.columns:
    ds['pct_en_estrato_mpa'] = ds.groupby('Estrato')['meses_para_agotar'].rank(pct=True) * 100.0

# Histograma de meses_para_agotar
vals = ds['meses_para_agotar'].dropna()
if not vals.empty:
    vals_plot = vals.clip(lower=vals.quantile(0.01), upper=vals.quantile(0.99))
    plt.figure()
    vals_plot.hist(bins=30)
    plt.title("Distribución de meses para agotar (proyectos)")
    plt.xlabel("Meses"); plt.ylabel("Frecuencia")
    plt.tight_layout(); plt.show()



## 9. ¿Qué variables influyen en el éxito? (modelo explicativo)

Entrenamos un **RandomForestRegressor** para predecir `meses_para_agotar` **sin usar** variables que *definen* ese target (como `Un. Disponible Proyecto` o `Ventas Prom.`).  
Incluimos: ubicación (Zona/SubZona/Barrio), producto/precio (área y $/m² agregados y de proyecto), tamaño (`Tot. Un. Proyecto`) y estado (`Estado Etapas`).

> Nota: Para evitar fuga, excluimos explícitamente `Un. Disponible Proyecto` y `Ventas Promedio` de los features.


In [ ]:

model_df = ds.copy()

# Selección de features (ajusta si cambian nombres)
num_feats = [c for c in [
    '_ppm2_median','_ppm2_mean','_area_median','_area_mean','_precio_median','_precio_mean',
    col_precio_m2, col_precio_p, col_area_p, col_tot_un, col_estrato, 'n_unidades'
] if c in model_df.columns]

cat_feats = [c for c in [col_zona, col_subzona, col_barrio, col_estado] if c and c in model_df.columns]

# Filtrado de filas con target
model_df = model_df.dropna(subset=['meses_para_agotar']).copy()

# Limpieza numérica
for c in num_feats:
    model_df[c] = pd.to_numeric(model_df[c], errors='coerce')

# One-hot para categóricas
X_cat = pd.get_dummies(model_df[cat_feats].astype(str), dummy_na=True, prefix=cat_feats)
X = pd.concat([model_df[num_feats].fillna(0).reset_index(drop=True),
               X_cat.reset_index(drop=True)], axis=1)
y = model_df['meses_para_agotar'].values

# Entrenamiento simple + CV rápido (MAE)
kf = KFold(n_splits=5, shuffle=True, random_state=42)
maes = []
for tr, te in kf.split(X, y):
    rf = RandomForestRegressor(n_estimators=250, random_state=42, n_jobs=-1, max_depth=None, min_samples_leaf=5)
    rf.fit(X.iloc[tr], y[tr])
    pred = rf.predict(X.iloc[te])
    mae = np.mean(np.abs(pred - y[te]))
    maes.append(mae)

print("CV MAE (meses):", np.round(np.mean(maes), 2), "±", np.round(np.std(maes), 2))

# Entrenar final para importancias
rf = RandomForestRegressor(n_estimators=500, random_state=42, n_jobs=-1, max_depth=None, min_samples_leaf=5)
rf.fit(X, y)

importancias = pd.DataFrame({
    "feature": X.columns,
    "importance": rf.feature_importances_
}).sort_values("importance", ascending=False)

# Agregar importancias por variable original (suma de dummies)
def var_origen(f):
    for base in cat_feats:
        if f.startswith(base + "_"):
            return base
    return f

importancias["variable_origen"] = importancias["feature"].apply(var_origen)
imp_group = importancias.groupby("variable_origen")["importance"].sum().reset_index().sort_values("importance", ascending=False)

display(imp_group.head(15))

# Gráfico simple del Top-15
top15 = imp_group.head(15)
plt.figure()
plt.barh(top15["variable_origen"][::-1], top15["importance"][::-1])
plt.xlabel("Importancia (suma de dummies)"); plt.title("Top-15 variables que explican velocidad")
plt.tight_layout(); plt.show()



## 10. Exportables


In [ ]:

# Guardamos los resultados principales
out1 = Path('clasificacion_proyectos_exito.csv')
ds.to_csv(out1, index=False)

out2 = Path('importancias_variables.csv')
imp_group.to_csv(out2, index=False)

print("Generados:")
print(" -", out1.resolve())
print(" -", out2.resolve())



## 11. Siguientes pasos (para acercarnos más a tus **escenarios**)

- Si cuentas con **fecha de venta por unidad** en *Inmuebles*, construye la **serie mensual** por proyecto:
  - *Tiempo al 50%* y *80%* vendido, **front‑loading** (concentración temprana), **tiempo real a sold‑out**.
- Normaliza por **segmento** (Zona/SubZona/Estrato/VIS) usando percentiles por grupo para comparar “manzanas con manzanas”.
- Usa **validación temporal** si el objetivo es predecir éxito al lanzar (evita usar información futura).
- Explora **elasticidad precio‑ventas** para simulación de escenarios de pricing.
